# Install cuplyr on Google Colab

GPU-accelerated dplyr for R, installed in ~5 minutes.

## Before running:
1. **Runtime → Change runtime type**
2. Set **Runtime type = R**, **Hardware accelerator = T4 GPU**
3. Click **Save**, then **Run all**

## Step 1: Pre-flight check

In [ ]:
# Verify GPU is attached
gpu_check <- system('nvidia-smi -L', intern = TRUE)
if (length(gpu_check) == 0 || grepl('error|fail', gpu_check[1], ignore.case = TRUE)) {
  stop('No GPU detected! Go to Runtime -> Change runtime type -> GPU')
}
cat('GPU:', gpu_check[1], '\n')

# Find CUDA
for (p in c('/usr/local/cuda', '/usr/local/cuda-12.8', '/usr/local/cuda-12.4', '/usr/local/cuda-12')) {
  if (file.exists(file.path(p, 'include', 'cuda.h'))) {
    Sys.setenv(CUDA_HOME = p)
    break
  }
}
cat('CUDA:', Sys.getenv('CUDA_HOME'), '\n')

# Set up C++20 for R
r_makevars <- path.expand('~/.R/Makevars')
dir.create(dirname(r_makevars), showWarnings = FALSE, recursive = TRUE)
writeLines(c('CXX20=g++', 'CXX20STD=-std=gnu++20', 'CXX20FLAGS=-O2 -fPIC'), r_makevars)

## Step 2: Install mamba (package manager for RAPIDS)

In [ ]:
# Install miniforge (provides mamba)
miniforge_dir <- '/opt/miniforge'
mamba <- file.path(miniforge_dir, 'bin', 'mamba')

if (!file.exists(mamba)) {
  cat('Installing Miniforge...\n')
  system('wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O /tmp/miniforge.sh')
  system(sprintf('bash /tmp/miniforge.sh -b -p %s 2>&1', miniforge_dir))
}

stopifnot('Miniforge install failed' = file.exists(mamba))

# Add mamba to PATH so install_cuplyr() can find it
old_path <- Sys.getenv('PATH')
Sys.setenv(PATH = paste(file.path(miniforge_dir, 'bin'), old_path, sep = ':'))
cat('mamba ready\n')

## Step 3: Install cuplyr

This clones the `install` branch of cuplyr, installs RAPIDS via conda, and builds the package. Takes 3-5 minutes on first run.

In [ ]:
# Clone cuplyr install branch from GitHub
repo_dir <- '/content/cuplyr'
repo_branch <- 'install'
if (dir.exists(repo_dir)) unlink(repo_dir, recursive = TRUE)

status <- system2('git', c('clone', '--depth', '1', '-b', repo_branch,
                           'https://github.com/bbtheo/cuplyr.git', repo_dir),
                  stdout = FALSE, stderr = FALSE)
if (status != 0) {
  stop('Git clone failed. Check network connection or branch name.', call. = FALSE)
}

# Source the install functions
source(file.path(repo_dir, 'R', 'check-deps.R'))
source(file.path(repo_dir, 'R', 'install.R'))

# Run installer with Colab-specific settings
Sys.setenv(CUPLYR_ENV = 'colab')
install_cuplyr(
  repo = repo_dir,
  method = 'conda',
  conda_prefix = '/opt/rapids',
  verbose = TRUE
)

## Step 4: Verify installation

This configures loader paths once, then loads cuplyr normally in the notebook session.

In [ ]:
# CRITICAL: Verify conda libstdc++ has the required GLIBCXX version
cat('Checking conda libstdc++...\n')
conda_stdc <- '/opt/rapids/lib/libstdc++.so.6'
if (!file.exists(conda_stdc)) {
  stop('RAPIDS libstdc++ not found at: ', conda_stdc, '\n',
       'RAPIDS installation may have failed in Step 3.', call. = FALSE)
}

# Check for GLIBCXX_3.4.31 (use grep with -a to treat as text, strings may not be installed)
glibcxx_check <- system2('grep', c('-ao', 'GLIBCXX_3.4.31', conda_stdc),
                         stdout = TRUE, stderr = FALSE)
if (length(glibcxx_check) == 0 || !any(grepl('GLIBCXX_3.4.31', glibcxx_check))) {
  stop('RAPIDS libstdc++ does not contain GLIBCXX_3.4.31.\n',
       'The RAPIDS environment may be outdated or incomplete.\n',
       'This usually means conda installed an older version.', call. = FALSE)
}
cat('  ✓ GLIBCXX_3.4.31 found in conda libstdc++\n')

# Configure dynamic linker: write config file and run ldconfig
cat('Configuring dynamic linker...\n')
conf_cmd <- "printf '/opt/rapids/lib\\n/usr/local/nvidia/lib64\\n' > /etc/ld.so.conf.d/00-cuplyr-rapids.conf && ldconfig"
ldconfig_status <- system2('bash', c('-c', conf_cmd), stdout = TRUE, stderr = TRUE)
ldconfig_exit <- attr(ldconfig_status, 'status')
if (is.null(ldconfig_exit)) ldconfig_exit <- 0L

if (ldconfig_exit != 0L) {
  stop('ldconfig setup failed (exit code: ', ldconfig_exit, ').\n',
       'Output: ', paste(ldconfig_status, collapse = '\n'),
       call. = FALSE)
}

# Verify the config file was written
if (!file.exists('/etc/ld.so.conf.d/00-cuplyr-rapids.conf')) {
  stop('Failed to write linker config file. Check permissions.', call. = FALSE)
}
cat('  ✓ Wrote /etc/ld.so.conf.d/00-cuplyr-rapids.conf\n')

# Verify ldconfig registered RAPIDS lib
ldconfig_verify <- system2('ldconfig', c('-p'), stdout = TRUE, stderr = FALSE)
rapids_libs <- grep('/opt/rapids/lib/', ldconfig_verify, value = TRUE)
if (length(rapids_libs) == 0) {
  warning('RAPIDS lib not visible in ldconfig cache. Will rely on LD_LIBRARY_PATH.')
} else {
  cat('  ✓ ldconfig cache updated (', length(rapids_libs), ' RAPIDS libraries registered)\n')
}

# Set environment variables with RAPIDS lib FIRST (critical for libstdc++ resolution)
cat('\nSetting LD_LIBRARY_PATH...\n')
old_ld <- Sys.getenv('LD_LIBRARY_PATH')
new_ld <- paste('/opt/rapids/lib', '/usr/local/nvidia/lib64', old_ld, sep = ':')
Sys.setenv(
  LD_LIBRARY_PATH = new_ld,
  R_LD_LIBRARY_PATH = new_ld
)

# Show what will be searched first
ld_parts <- strsplit(new_ld, ':', fixed = TRUE)[[1]]
cat('  First 3 search paths:\n')
cat('    1. ', ld_parts[1], '\n')
cat('    2. ', ld_parts[2], '\n')
if (length(ld_parts) >= 3) cat('    3. ', ld_parts[3], '\n')
cat('\n')

# Load cuplyr
cat('Loading cuplyr package...\n')
tryCatch({
  library(cuplyr)
  cat('  ✓ Package loaded successfully!\n\n')
}, error = function(e) {
  cat('\n✗ Package load failed:\n')
  cat('  ', conditionMessage(e), '\n\n')
  cat('Diagnostic commands to run:\n')
  cat('  SO=/usr/local/lib/R/site-library/cuplyr/libs/cuplyr.so\n')
  cat('  ldd $SO | grep libstdc\n')
  cat('  readelf -d $SO | grep RUNPATH\n\n')
  stop('Could not load cuplyr. See diagnostic commands above.', call. = FALSE)
})

# Verify installation
verify_installation()

## Step 5: Try cuplyr

GPU-accelerated dplyr — same syntax, runs on GPU.

In [ ]:
# Filter, mutate, select, arrange
tbl_gpu(mtcars) |>
  filter(mpg > 20) |>
  mutate(kpl = mpg * 0.425) |>
  select(mpg, kpl, cyl, hp) |>
  arrange(desc(mpg)) |>
  collect()

In [ ]:
# Group by and summarise
tbl_gpu(mtcars) |>
  group_by(cyl) |>
  summarise(
    count = n(),
    avg_mpg = mean(mpg),
    avg_hp = mean(hp)
  ) |>
  arrange(cyl) |>
  collect()

In [ ]:
# Lazy evaluation with query optimization
tbl_gpu(mtcars, lazy = TRUE) |>
  filter(mpg > 15) |>
  mutate(power_weight = hp / wt) |>
  group_by(cyl) |>
  summarise(avg_pw = mean(power_weight)) |>
  arrange(desc(avg_pw)) |>
  collect()

## Troubleshooting

Run `diagnostics()` if GPU is not detected — paste the output in a [GitHub issue](https://github.com/bbtheo/cuplyr/issues).

In [ ]:
# Full diagnostics — paste this output in bug reports
diagnostics(redact = FALSE)

# Dependency check
check_deps()